In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path='training_set.csv'

In [4]:
df=pd.read_csv(path)
print(df)

      Landmark_1  Landmark_1.1  Landmark_2  Landmark_2.1  Landmark_3  \
0       0.711043      0.780427    0.659234      0.727291    0.649127   
1       0.708050      0.781900    0.701195      0.714222    0.697141   
2       0.703424      0.780437    0.710349      0.712303    0.707363   
3       0.690665      0.769731    0.708838      0.695502    0.704932   
4       0.679726      0.768610    0.701220      0.697111    0.697889   
...          ...           ...         ...           ...         ...   
1883    0.713533      0.754582    0.716890      0.665118    0.720678   
1884    0.474237      0.820640    0.488393      0.728018    0.537454   
1885    0.435381      0.848009    0.452731      0.757673    0.502674   
1886    0.419285      0.575615    0.412108      0.477879    0.382376   
1887    0.459278      0.560038    0.438118      0.454594    0.405663   

      Landmark_3.1  Landmark_4  Landmark_4.1  Landmark_5  Landmark_5.1  ...  \
0         0.666429    0.643421      0.620828    0.637780

In [5]:
# Print unique values and their counts in the "Label" column
label_counts = df['Label'].value_counts()

for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")


Label: land, Count: 276
Label: left, Count: 275
Label: backward, Count: 261
Label: flip, Count: 248
Label: up, Count: 233
Label: down, Count: 204
Label: right, Count: 199
Label: forward, Count: 192


In [6]:
from sklearn.model_selection import train_test_split

# Define the feature columns (excluding the 'Label' column)
feature_columns = df.columns[df.columns != 'Label']

# Initialize lists to store train and test sets
train_data = []
test_data = []

# Split the data for each unique label
for label in df['Label'].unique():
    # Select data for the current label
    label_data = df[df['Label'] == label]

    # Split the data into 90% for training and 10% for testing
    train_label, test_label = train_test_split(label_data, test_size=0.1, random_state=42)

    # Append to the train and test sets
    train_data.append(train_label)
    test_data.append(test_label)

# Concatenate the train and test sets for each label
train_df = pd.concat(train_data)
test_df = pd.concat(test_data)

# Display the shapes of train and test sets
print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)


Train set shape: (1695, 43)
Test set shape: (193, 43)


In [7]:
# Assuming 'Label' is the column to be dropped
x_train = train_df.drop(columns='Label')
x_test = test_df.drop(columns='Label')
y_train = train_df['Label']
y_test = test_df['Label']


In [8]:
print(y_test)
print(y_train)

30      backward
143     backward
170     backward
73      backward
159     backward
          ...   
1679          up
1862          up
1665          up
1793          up
1841          up
Name: Label, Length: 193, dtype: object
46      backward
158     backward
15      backward
237     backward
229     backward
          ...   
1761          up
1669          up
1747          up
1834          up
1757          up
Name: Label, Length: 1695, dtype: object


In [9]:
print(x_train)
print(x_test)

      Landmark_1  Landmark_1.1  Landmark_2  Landmark_2.1  Landmark_3  \
46      0.755237      0.798447    0.693529      0.739575    0.680238   
158     0.690413      0.548448    0.649843      0.528170    0.638383   
15      0.669146      0.728127    0.630494      0.664610    0.623117   
237     0.152070      0.387291    0.182219      0.369702    0.198906   
229     0.208370      0.937315    0.216483      0.872388    0.199270   
...          ...           ...         ...           ...         ...   
1761    0.691856      0.863727    0.678306      0.772705    0.653390   
1669    0.368657      0.667468    0.346659      0.552859    0.311714   
1747    0.786952      0.917946    0.766721      0.819830    0.757668   
1834    0.308823      0.814156    0.327682      0.690792    0.326303   
1757    0.695969      0.593263    0.651356      0.503459    0.617305   

      Landmark_3.1  Landmark_4  Landmark_4.1  Landmark_5  Landmark_5.1  ...  \
46        0.671758    0.669077      0.628551    0.656666

In [10]:
# Create a mapping dictionary
label_mapping = {'up': 0, 'down': 1, 'left': 2, 'right': 3, 'land': 4, 'flip': 5, 'forward': 6, 'backward': 7}

# Apply the mapping to training set labels
y_train_encoded = y_train.map(label_mapping)

# Apply the mapping to test set labels
y_test_encoded = y_test.map(label_mapping)


In [11]:

unique_values_counts = y_test_encoded.value_counts()
for value, count in unique_values_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 4, Count: 28
Value: 2, Count: 28
Value: 7, Count: 27
Value: 5, Count: 25
Value: 0, Count: 24
Value: 1, Count: 21
Value: 6, Count: 20
Value: 3, Count: 20


In [22]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=50)

# Train the model
xgb_classifier.fit(x_train, y_train_encoded)


y_pred_train = xgb_classifier.predict(x_train)

# Evaluate the accuracy on the training set
accuracy_train = accuracy_score(y_train_encoded, y_pred_train)
print(f"Training Accuracy: {accuracy_train}")

Training Accuracy: 1.0


In [24]:
from sklearn.metrics import accuracy_score
y_pred = xgb_classifier.predict(x_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9896373056994818


In [25]:
import pickle

# Save the trained model to a file
model_filename = "xgb_model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(xgb_classifier, model_file)

print(f"Model saved to {model_filename}")

Model saved to xgb_model.pkl


In [68]:
def get_landmarks(image_path):
    # Read the image
    img = cv2.imread(image_path)
    x, y, c = img.shape

    # Initialize Mediapipe hands
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    mpDraw = mp.solutions.drawing_utils

    # Convert the image to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Get hand landmark prediction
    result = hands.process(img_rgb)

    # Extract landmarks
    landmarks = []
    if result.multi_hand_landmarks:
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                landmarks.append([lm.x, lm.y])

    return landmarks

In [19]:
import cv2
import mediapipe as mp

In [70]:
def predict_label_and_probs(image_path, model, label_mapping):
    # Get landmarks from the image
    landmarks = get_landmarks(image_path)
    
    # Convert landmarks to a flat list
    flat_landmarks = [val for sublist in landmarks for val in sublist]

    # Reshape the flat list to match the input format expected by the model
    input_data = np.array(flat_landmarks).reshape(1, -1)

    # Make prediction using the loaded XGBoost model
    predicted_probs = model.predict_proba(input_data)[0]

    # Convert the encoded label back to the original label using the mapping dictionary
    predicted_labels = [key for key, value in label_mapping.items()]
    
    # Print the predicted label and probabilities
    print("Predicted Label Probabilities:")
    for label, prob in zip(predicted_labels, predicted_probs):
        print(f"{label}: {prob:.2%}")

# Example usage
image_path = "gesture_dataset/backward_838.png"
predict_label_and_probs(image_path, xgb_classifier, label_mapping)

ValueError: Feature shape mismatch, expected: 42, got 0